In [1]:
import numpy as np

from scipy.spatial.transform import Rotation as R

from reachy_kinematics.orbita import Actuator

In [2]:
def inverse(roll, pitch, yaw):
    a = Actuator(R0=np.eye(3))
    q = R.from_euler('xyz', [roll, pitch, yaw]).as_quat()
    return np.deg2rad(a.get_angles_from_quaternion(q[3], q[0], q[1], q[2]))

In [3]:
N = int(120 * 1 + 1)
print(N ** 3)

X, Y = [], []

space = np.deg2rad(np.linspace(-60, 60, N))
yaw_space = np.deg2rad(np.linspace(-120, 120, 2 * N))


for roll in space:
    for pitch in space:
        for yaw in space:
            try:
                d = inverse(roll, pitch, yaw)
                X.append(d)
                Y.append((roll, pitch, yaw))
            except ValueError:
                pass

X = np.array(X)
Y = np.array(Y)

X.shape, Y.shape


1771561


((1148895, 3), (1148895, 3))

In [4]:
np.rad2deg(X), np.rad2deg(Y)

(array([[-176.18674615,  -74.56476751,   25.49214703],
        [-175.18674615,  -73.56476751,   26.49214703],
        [-174.18674615,  -72.56476751,   27.49214703],
        ...,
        [ 104.08431744,   43.29390981,  -15.6796107 ],
        [ 105.08431744,   44.29390981,  -14.6796107 ],
        [ 106.08431744,   45.29390981,  -13.6796107 ]]),
 array([[-58., -26., -60.],
        [-58., -26., -59.],
        [-58., -26., -58.],
        ...,
        [ 58.,  26.,  58.],
        [ 58.,  26.,  59.],
        [ 58.,  26.,  60.]]))

In [5]:
np.save(f'X-{N}.npy', X)
np.save(f'Y-{N}.npy', Y)

In [6]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=5)

In [7]:
knn.fit(X, Y)

KNeighborsRegressor()

In [8]:
e = []

for _ in range(1000):
    try:
        roll, pitch, yaw = np.deg2rad(np.random.rand(3) * 120 - 60)
        y = np.array([[roll, pitch, yaw]])

        disks = inverse(roll, pitch, yaw)
        x = np.array([disks])

        y_hat = knn.predict(x)

        e.append(np.linalg.norm(y - y_hat))
    except ValueError:
        pass

e = np.array(e)

In [9]:
np.rad2deg(e).max()

1.129446959016913

In [10]:
e.shape

(645,)

In [11]:
import pickle

with open(f'forward-knn-{N}.pkl', 'wb') as f:
    pickle.dump(knn, f)